# 

In [1]:
import psutil    #A

# Collect CPU, Memory, and Disk usage data
cpu_usage = psutil.cpu_percent(interval=1)    #B
memory_usage = psutil.virtual_memory().percent    #C
disk_usage = psutil.disk_usage('/').percent    #D

# Print collected metrics
print(f"CPU Usage: {cpu_usage}%")    #E
print(f"Memory Usage: {memory_usage}%")    #E
print(f"Disk Usage: {disk_usage}%")    #E


CPU Usage: 31.1%
Memory Usage: 65.1%
Disk Usage: 34.5%


In [19]:
%%bash

# Check OS type
os_type=$(uname)  #A

# Collect CPU usage based on OS
if [[ "$os_type" == "Linux" ]]; then
    cpu_usage=$(top -b -n1 | awk '/Cpu\(s\)/ {print 100 - $8}')  #B
elif [[ "$os_type" == "Darwin" ]]; then  # macOS
    cpu_usage=$(ps -A -o %cpu | awk '{s+=$1} END {print s}')  #C
else
    cpu_usage=$(vmstat 1 2 | tail -1 | awk '{print 100 - $15}')  #D
fi

# Collect Disk Usage
disk_usage=$(df -h | grep '^/dev/' | awk '{ print $5 " " $1 }')  #E

# Print collected metrics
echo "Operating System: $os_type"  #F
echo "CPU Usage: $cpu_usage%"  #G
echo "Disk Usage:"  #H
echo "$disk_usage"  #I


Operating System: Darwin
CPU Usage: 177%
Disk Usage:
2% /dev/disk3s1s1
1% /dev/disk3s6
2% /dev/disk3s2
1% /dev/disk3s4
2% /dev/disk2s2
2% /dev/disk2s1
1% /dev/disk2s3
34% /dev/disk3s5
1% /dev/disk3s3


In [11]:
import subprocess  # A
import json  # B
import pandas as pd  # C
from IPython.display import display  # D

# PowerShell commands
cpu_cmd = "Get-WmiObject Win32_Processor | Measure-Object -Property LoadPercentage -Average | Select-Object -ExpandProperty Average"  # E
memory_cmd = "Get-WmiObject Win32_OperatingSystem | Select-Object @{Name='FreeMemory';Expression={[math]::round($_.FreePhysicalMemory/1MB,2)}},TotalVisibleMemorySize | ConvertTo-Json -Compress"  # F
disk_cmd = "Get-WmiObject Win32_LogicalDisk -Filter 'DeviceID=\"C:\"' | Select-Object DeviceID, @{Name='FreeSpaceGB';Expression={[math]::round($_.FreeSpace/1GB,2)}} | ConvertTo-Json -Compress"  # G

# Run PowerShell commands and capture output
cpu_usage = subprocess.run(["powershell", "-Command", cpu_cmd], capture_output=True, text=True).stdout.strip()  # H
memory_output = subprocess.run(["powershell", "-Command", memory_cmd], capture_output=True, text=True).stdout.strip()  # I
disk_output = subprocess.run(["powershell", "-Command", disk_cmd], capture_output=True, text=True).stdout.strip()  # J

# Parse JSON output
memory_usage = json.loads(memory_output) if memory_output else {"FreeMemory": "N/A", "TotalVisibleMemorySize": "N/A"}  # K
disk_usage = json.loads(disk_output) if disk_output else {"FreeSpaceGB": "N/A"}  # L

# Convert results into a DataFrame for Jupyter Notebook display
data = {  # M
    "Metric": ["CPU Usage", "Free Memory (MB)", "Total Memory (MB)", "Free Disk Space (GB)"],
    "Value": [cpu_usage + "%", memory_usage.get("FreeMemory", "N/A"), memory_usage.get("TotalVisibleMemorySize", "N/A"), disk_usage.get("FreeSpaceGB", "N/A")]
}

df = pd.DataFrame(data)  # N

# Display DataFrame in Jupyter Notebook
display(df)  # O



,Metric,Value
0,CPU Usage,8%
1,Free Memory (MB),51.65
2,Total Memory (MB),67016960
3,Free Disk Space (GB),1617.3
